<center><h1><b>Algoritmo Genético</b><h1></center>

O primeiro passo é criar uma população de bitstrings aleatórias.

Poderíamos usar valores booleanos True e False, valores de string "0" e "1", ou valores inteiros 0 e 1.

Neste caso, usaremos valores inteiros.

In [37]:
from numpy.random import randint

# bits
n_bits = 16
# define o tamanho da população
n_pop = 10

# população inicial de bitstring aleatórias
pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]

pop

[[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
 [1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1],
 [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1],
 [1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]]

Em seguida, podemos enumerar um número fixo de iterações do algoritmo, neste caso, controlado por um hiperparâmetro chamado “n_iter“.

In [54]:
# define o número total de iterações
n_iter = 10

# enumera as gerações
for gen in range(n_iter):
    # escreve a geração atual
    print(f"Geração {gen+1}")

Geração 1
Geração 2
Geração 3
Geração 4
Geração 5
Geração 6
Geração 7
Geração 8
Geração 9
Geração 10


O primeiro passo na iteração do algoritmo é avaliar todas as soluções candidatas.

Usaremos uma função chamada objective() como uma função objetivo genérica e a chamaremos para obter uma pontuação de aptidão, que minimizaremos.

In [48]:
# Avalia o fitness de todas as soluções na população
scores = [objective(c) for c in pop]

Podemos então selecionar os pais que serão usados ​​para criar os filhos.

O procedimento de seleção de torneio pode ser implementado como uma função que pega a população e retorna um pai selecionado.

O valor k é fixado em 3 com um argumento padrão, mas você pode experimentar com valores diferentes se quiser.

In [45]:
# seleção por torneio
def selection(pop, scores, k=3):
	# primeira seleção aleatória
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# checa se é melhor (ex. performa um torneio)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

Podemos então chamar essa função uma vez para cada posição na população para criar uma lista de pais.

In [49]:
# seleciona os pais
selected = [selection(pop, scores) for _ in range(n_pop)]

Podemos então criar a próxima geração.

Isso primeiro requer uma função para executar o crossover. Esta função pegará dois pais e a taxa de crossover. A taxa de crossover é um hiperparâmetro que determina se o crossover é executado ou não, e se não, os pais são copiados para a próxima geração. É uma probabilidade e normalmente tem um valor grande próximo a 1,0.

A função crossover() abaixo implementa o crossover usando um sorteio de um número aleatório no intervalo [0,1] para determinar se o crossover é executado, então selecionando um ponto de divisão válido se o crossover for executado.

In [33]:
from numpy.random import rand

# crossover entre dois pais para criar dois filhos
def crossover(p1, p2, r_cross):
	# filhos são cópias dos pais por padrão
	c1, c2 = p1.copy(), p2.copy()
	# checagem para a recombinação
	if rand() < r_cross:
		# seleciona o ponto de crossover excluindo o final da string
		pt = randint(1, len(p1)-2)
		# performa o crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

Também precisamos de uma função para executar mutação.

Este procedimento simplesmente inverte bits com uma probabilidade baixa controlada pelo hiperparâmetro “r_mut”.

In [ ]:
# operador de mutação
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# checagem para a mutação
		if rand() < r_mut:
			# troca o bit
			bitstring[i] = 1 - bitstring[i]

Podemos então percorrer a lista de pais e criar uma lista de filhos para serem usados ​​como a próxima geração, chamando as funções de cruzamento e mutação conforme necessário.

In [14]:
# taxa de crossover
r_cross = 0.9
# taxa de mutação
r_mut = 1.0 / float(n_bits)

# cria a próxima geração
children = list()
for i in range(0, n_pop, 2):
	# obtem os pais selecionados em pares
	p1, p2 = selected[i], selected[i+1]
	# crossover e mutação
	for c in crossover(p1, p2, r_cross):
		# mutação
		mutation(c, r_mut)
		# armazena para a próxima geração
		children.append(c)

Podemos unir tudo isso em uma função chamada genetic_algorithm() que pega o nome da função objetivo e os hiperparâmetros da pesquisa e retorna a melhor solução encontrada durante a pesquisa.

In [47]:
# algoritmo genético
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(pop[0])
	# enumerate generations
	for gen in range(n_iter):
		# evaluate all candidates in the population
		scores = [objective(c) for c in pop]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

<center><h2><b>Genetic Algorithm for OneMax</b></h2></center>

Nesta seção, aplicaremos o algoritmo genético a um problema de otimização baseado em string binária.

O problema é chamado OneMax e avalia uma string binária com base no número de 1s na string. Por exemplo, uma bitstring com um comprimento de 20 bits terá uma pontuação de 20 para uma string de todos os 1s.

Dado que implementamos o algoritmo genético para minimizar a função objetivo, podemos adicionar um sinal negativo a essa avaliação para que grandes valores positivos se tornem grandes valores negativos.

A função onemax() abaixo implementa isso e pega uma bitstring de valores inteiros como entrada e retorna a soma negativa dos valores.

In [50]:
# objective function
def onemax(x):
	return -sum(x)

Em seguida, podemos configurar a busca.

A pesquisa será executada por 100 iterações e usaremos 20 bits em nossas soluções candidatas, o que significa que a aptidão ótima será -20,0.

O tamanho da população será 100, e usaremos uma taxa de cruzamento de 90 por cento e uma taxa de mutação de 5 por cento. Esta configuração foi escolhida após um pouco de tentativa e erro.

In [ ]:
...
# define the total iterations
n_iter = 100
# bits
n_bits = 20
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)

Ligando tudo isso, o exemplo completo da aplicação do algoritmo genético à função objetivo OneMax está listado abaixo.

In [51]:
# genetic algorithm search of the one max optimization problem
from numpy.random import randint
from numpy.random import rand

# objective function
def onemax(x):
	return -sum(x)

# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(pop[0])
	# enumerate generations
	for gen in range(n_iter):
		# evaluate all candidates in the population
		scores = [objective(c) for c in pop]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

# define the total iterations
n_iter = 100
# bits
n_bits = 20
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)
# perform the genetic algorithm search
best, score = genetic_algorithm(onemax, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>0, new best f([0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]) = -13.000
>0, new best f([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]) = -14.000
>1, new best f([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]) = -15.000
>1, new best f([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]) = -16.000
>3, new best f([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]) = -18.000
>5, new best f([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -19.000
>8, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000
Done!
f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000000


<center><h2><b>Genetic Algorithm for Continuous Function Optimization</b><h2></center>

Otimizar a função OneMax não é muito interessante; é mais provável que queiramos otimizar uma função contínua.

Por exemplo, podemos definir a função de minimização x^2 que pega variáveis ​​de entrada e tem um ótimo em f(0, 0) = 0,0.

In [52]:
# objective function
def objective(x):
	return x[0]**2.0 + x[1]**2.0

Podemos minimizar essa função com um algoritmo genético.

Primeiro, precisamos definir os limites de cada variável de entrada.

In [ ]:
...
# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

Tomaremos o hiperparâmetro “n_bits” como um número de bits por variável de entrada para a função objetivo e o definiremos como 16 bits.

In [ ]:
...
# bits per variable
n_bits = 16

Isso significa que nossa sequência de bits real terá (16 * 2) = 32 bits, dadas as duas variáveis ​​de entrada.

Devemos atualizar nossa taxa de mutação de acordo.

In [ ]:
...
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))

Em seguida, precisamos garantir que a população inicial crie sequências de bits aleatórias que sejam grandes o suficiente.

In [ ]:
...
# initial population of random bitstring
pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]

Por fim, precisamos decodificar as bitstrings para números antes de avaliar cada uma com a função objetivo.

Podemos conseguir isso primeiro decodificando cada substring para um inteiro, depois escalando o inteiro para o intervalo desejado. Isso dará um vetor de valores no intervalo que pode então ser fornecido para a função objetivo para avaliação.

A função decode() abaixo implementa isso, pegando os limites da função, o número de bits por variável e uma bitstring como entrada e retorna uma lista de valores reais decodificados.

In [ ]:
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded

Podemos então chamar isso no início do loop do algoritmo para decodificar a população e, então, avaliar a versão decodificada da população.

In [ ]:
...
# decode population
decoded = [decode(bounds, n_bits, p) for p in pop]
# evaluate all candidates in the population
scores = [objective(d) for d in decoded]

Ligando tudo isso, o exemplo completo do algoritmo genético para otimização de função contínua está listado abaixo.

In [53]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

# objective function
def objective(x):
	return x[0]**2.0 + x[1]**2.0

# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded

# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	# enumerate generations
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([4.981689453125, 1.60308837890625]) = 27.387122
>0, new best f([-1.647186279296875, 4.1046142578125]) = 19.561081
>0, new best f([-4.3408203125, 0.2996826171875]) = 18.932531
>0, new best f([3.14422607421875, -1.528778076171875]) = 12.223320
>0, new best f([-0.3668212890625, -0.787506103515625]) = 0.754724
>0, new best f([-0.00640869140625, -0.577850341796875]) = 0.333952
>1, new best f([-0.26397705078125, -0.3192138671875]) = 0.171581
>2, new best f([0.083770751953125, 0.2667236328125]) = 0.078159
>4, new best f([-0.00640869140625, -0.12542724609375]) = 0.015773
>5, new best f([-0.00640869140625, -0.0128173828125]) = 0.000205
>12, new best f([-0.00244140625, -0.010833740234375]) = 0.000123
>13, new best f([-0.003204345703125, 0.007781982421875]) = 0.000071
>16, new best f([-0.00244140625, -0.002593994140625]) = 0.000013
>18, new best f([-0.00244140625, -0.001373291015625]) = 0.000008
>20, new best f([-0.00244140625, -0.000152587890625]) = 0.000006
>20, new best f([-0.00